In [149]:
!pip install bert-for-tf2
!pip install sentencepiece

  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30534 sha256=dc6192de6da0e0c64fe9ac5ef04dc974fd8b906cc61e0c387cbce6d7183719ce
  Stored in directory: c:\users\steff\appdata\local\pip\cache\wheels\6f\c7\91\f2b2c2b3cec30578c5de7c27ac99659a2013501dd66e7e3db0
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19473 sha256=6192d20599693fef32fc1510ad5b7bb4325acb8c33f9b297d3ff0f07d7675fa8
  Stored in directory: c:\users\steff\appdata\local\pip\cache\wheels\be\17\6c\5c924411a614ee0a74b2dc4f04c9e61dacc4e60fe9854f4f70
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7912 sha256=6e98b6b1b77bd0156270366c4119fa7991c4653b7f49ceb004cffdac471b0633
  Stored in directory: c:\users\steff\appdata\local\pip\cache\wheels\29\ff\b1\77192657c3311dcae256412a7f36f73b064ace9c98312f5347
Successfully built bert-for-tf2 params-flow py-params


In [268]:
import bert
import math
from tensorflow.keras import layers
from tensorflow import keras
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub

In [239]:
import pandas as pd

train_df = pd.read_csv("game_reviews/train_gr/train.csv")
test_df = pd.read_csv("game_reviews/test_gr/test.csv")

In [240]:
train_df[train_df['user_suggestion']==0].head(3)

,review_id,title,year,user_review,user_suggestion
8,9,Spooky's Jump Scare Mansion,2015.0,"Somewhere between light hearted, happy parody ...",0
91,92,Spooky's Jump Scare Mansion,2015.0,Has a few nice elements but overall quite bori...,0
93,94,Spooky's Jump Scare Mansion,2015.0,boring. jump scares have 0 effect on me. if yo...,0


In [241]:
train_df[train_df['user_suggestion']==1].head(3)

,review_id,title,year,user_review,user_suggestion
0,1,Spooky's Jump Scare Mansion,2016.0,I'm scared and hearing creepy voices. So I'll...,1
1,2,Spooky's Jump Scare Mansion,2016.0,"Best game, more better than Sam Pepper's YouTu...",1
2,3,Spooky's Jump Scare Mansion,2016.0,"A littly iffy on the controls, but once you kn...",1


In [242]:
train_df.groupby('user_suggestion').describe()

review_id                                                    \
                    count          mean          std  min      25%      50%   
user_suggestion                                                               
0                  7526.0  13863.676854  7423.974011  9.0  8478.50  15604.5   
1                  9968.0  11335.646669  7644.702443  1.0  4119.75  11208.5   

                                      year                                 \
                      75%      max   count         mean       std     min   
user_suggestion                                                             
0                20199.75  25534.0  7429.0  2016.580966  1.288100  2011.0   
1                17735.25  25539.0  9887.0  2016.243754  1.445782  2011.0   

                                                 
                    25%     50%     75%     max  
user_suggestion                                  
0                2016.0  2017.0  2018.0  2018.0  
1                2015.0  2016.0  2017.0  2018.0

In [243]:
#Remove title, id, year
train_df = train_df[['user_review','user_suggestion']]
train_df.head()

,user_review,user_suggestion
0,I'm scared and hearing creepy voices. So I'll...,1
1,"Best game, more better than Sam Pepper's YouTu...",1
2,"A littly iffy on the controls, but once you kn...",1
3,"Great game, fun and colorful and all that.A si...",1
4,Not many games have the cute tag right next to...,1


In [244]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [245]:
tokenizer.tokenize("don't be so judgmental")

['don', "'", 't', 'be', 'so', 'judgment', '##al']

In [246]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("dont be so judgmental"))

[2123, 2102, 2022, 2061, 8689, 2389]

In [247]:
def tokenize_reviews(text_reviews):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_reviews))

In [248]:
new_train['user_review'] = train_df['user_review'].apply(tokenize_reviews)

In [249]:
records = new_train.to_records(index=False)
tokenized_train = list(records)

In [275]:
tokenized_train = [(i[0], i[1]) for i in tokenized_train]

In [251]:
train_ds = tf.data.Dataset.from_generator(lambda: tokenized_train, output_types=(tf.int32, tf.int32))

In [252]:
train_ds

<FlatMapDataset shapes: (<unknown>, <unknown>), types: (tf.int32, tf.int32)>

In [253]:
BATCH_SIZE = 32
train_data = train_ds.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [254]:
next(iter(train_data))

(<tf.Tensor: shape=(32, 582), dtype=int32, numpy=
 array([[2220, 3229, 3319, ...,    0,    0,    0],
        [2220, 3229, 3319, ...,    0,    0,    0],
        [2220, 3229, 3319, ...,    0,    0,    0],
        ...,
        [2023, 2003, 2107, ...,    0,    0,    0],
        [4569, 2895, 1011, ...,    0,    0,    0],
        [2009, 3849, 2000, ...,    0,    0,    0]])>,
 <tf.Tensor: shape=(32,), dtype=int32, numpy=
 array([0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
        0, 0, 1, 1, 1, 1, 0, 1, 1, 0])>)

In [255]:
new_test['user_review'] = test_df['user_review'].apply(tokenize_reviews)
records = new_test.to_records(index=False)
tokenized_test = list(records)
tokenized_test = [(i[0], i[1]) for i in tokenized_train]
test_ds = tf.data.Dataset.from_generator(lambda: tokenized_test, output_types=(tf.int32, tf.int32))
test_data = test_ds.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))
next(iter(test_data))

(<tf.Tensor: shape=(32, 582), dtype=int32, numpy=
 array([[2220, 3229, 3319, ...,    0,    0,    0],
        [2220, 3229, 3319, ...,    0,    0,    0],
        [2220, 3229, 3319, ...,    0,    0,    0],
        ...,
        [2023, 2003, 2107, ...,    0,    0,    0],
        [4569, 2895, 1011, ...,    0,    0,    0],
        [2009, 3849, 2000, ...,    0,    0,    0]])>,
 <tf.Tensor: shape=(32,), dtype=int32, numpy=
 array([0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
        0, 0, 1, 1, 1, 1, 0, 1, 1, 0])>)

In [256]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [257]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 2

DROPOUT_RATE = 0.2

NB_EPOCHS = 5

In [258]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [259]:
text_model.compile(loss="binary_crossentropy",
                   optimizer="adam",
                   metrics=["accuracy"])

In [260]:
text_model.fit(train_data, epochs=NB_EPOCHS)

Epoch 1/5
383/383 [==============================] - 242s 621ms/step - loss: 0.4517 - accuracy: 0.7676
Epoch 2/5
383/383 [==============================] - 239s 624ms/step - loss: 0.1939 - accuracy: 0.9252
Epoch 3/5
383/383 [==============================] - 237s 620ms/step - loss: 0.0703 - accuracy: 0.9761
Epoch 4/5
383/383 [==============================] - 238s 622ms/step - loss: 0.0556 - accuracy: 0.9793
Epoch 5/5
383/383 [==============================] - 236s 618ms/step - loss: 0.0171 - accuracy: 0.9944


In [261]:
results = text_model.evaluate(test_data)
print(results)

383/383 [==============================] - 106s 276ms/step - loss: 0.0062 - accuracy: 0.9984
[0.006198831833899021, 0.998448371887207]


In [266]:
text_trial = ["This game is so sexy that I cried tears of joy while playing this game. I love it!", 
              "Worst game I have ever played. Do not buy it. So not worth the money.",
             "This game is okay I guess."]
text_trial = [tokenize_reviews(i) for i in text_trial]

In [267]:
print([text_model.predict([review]) for review in text_trial])

[array([[0.9999999]], dtype=float32), array([[0.00010345]], dtype=float32), array([[0.02982047]], dtype=float32)]


In [274]:
text_model.save('./')

INFO:tensorflow:Assets written to: ./assets


INFO:tensorflow:Assets written to: ./assets
